In [ ]:
import json

import requests


response = requests.get("https://crefnavigator.mitre.org/api/mitigations/enterprise", verify=False)

response.raise_for_status()

candidate_mitigations = response.json()

print(json.dumps(candidate_mitigations[0], indent=2))

with open("candidate-mitigations.json", "w") as f:
    json.dump(candidate_mitigations, f, indent=4)


In [ ]:
for candidate_mitigation in candidate_mitigations:
    detail_response = requests.get(f'https://crefnavigator.mitre.org/api/mitigations/{candidate_mitigation.get("id")}', verify=False)
    detail_response.raise_for_status()
    candidate_mitigation["associatedTechniques"] = detail_response.json()["associatedTechniques"]

with open("candidate-mitigation-details.json", "w") as f:
    json.dump(candidate_mitigations, f, indent=4)



In [ ]:
# invert

techniques = {}

for candidate_mitigation in candidate_mitigations:
    for associated_technique in candidate_mitigation.get("associatedTechniques", []):
        technique_id = associated_technique.get("technique_id")
        if technique_id not in techniques:
            techniques[technique_id] = {
                "candidate_mitigations": [],
                "control_list": []
            }
        techniques[technique_id]["candidate_mitigations"].append(candidate_mitigation["id"])
        techniques[technique_id]["control_list"].extend(associated_technique.get("controlList", []))

with open("technique_list.json", "w") as f:
    json.dump(techniques, f, indent=2)

